## Lecture 6  Part 2
## CUDA programming with Numba
### March 1, 2023


---
See [https://nyu-cds.github.io/python-numba/05-cuda/](https://nyu-cds.github.io/python-numba/05-cuda/).

---
If you graphics card support CUDA (Compute Unified Device Architecture) you must have a [CUDA driver](https://developer.nvidia.com/cuda-downloads) installed on your machine.


### Terminology
Several important terms in the topic of CUDA programming are:

**host**<br>
&nbsp;&nbsp;&nbsp;the CPU<br>
**host memory**<br>
&nbsp;&nbsp;&nbsp;the system main memory<br>
**device**<br>
&nbsp;&nbsp;&nbsp;the GPU<br>
**device memory**<br>
&nbsp;&nbsp;&nbsp;onboard memory on a GPU card<br>
**kernel**<br>
&nbsp;&nbsp;&nbsp;a GPU function launched by the host (CPU) and executed on the device (GPU)<br>
**device function**<br>
&nbsp;&nbsp;&nbsp;a GPU function executed on the device which can only be called from the device (i.e. from a kernel or another device function)


[https://devblogs.nvidia.com/easy-introduction-cuda-c-and-c/](https://devblogs.nvidia.com/easy-introduction-cuda-c-and-c/)

- The CUDA programming model is a heterogeneous model in which both the CPU and GPU are used. 

- In CUDA, the host refers to the CPU and its memory, while the device refers to the GPU and its memory. 

- Code run on the host can manage memory on both the host and device, and also launches kernels which are functions executed on the device. These kernels are executed by many GPU threads in parallel.


### CUDA Programming
Numba supports CUDA GPU programming by directly compiling a restricted subset of Python code into CUDA kernels and device functions following the CUDA execution model.

One feature that significantly simplifies writing GPU kernels is that Numba makes it appear that **the kernel has direct access to NumPy arrays**. NumPy arrays that are supplied as arguments to **the kernel are transferred between the CPU and the GPU automatically** (although this can also be an issue).

However, Numba does not implement the full CUDA API, so some features are not available. 

CUDA support in Numba is being actively developed, so eventually most of the features should be available.

### Device Management
It is possible to obtain a list of all the GPUs in the system 
using the following commands:

In [ ]:
# Listing the GPUs in the system
from numba import cuda
print(cuda.gpus)

If you do not have a CUDA-enabled GPU on your system, you will receive one of the following errors:
```
numba.cuda.cudadrv.error.CudaDriverError: CUDA initialized before forking
```
```
CudaSupportError: Error at driver init: 
[3] Call to cuInit results in CUDA_ERROR_NOT_INITIALIZED:
```
```
numba.cuda.cudadrv.error.CudaDriverError: Error at driver init:
CUDA disabled by user:
```
If you have a CUDA-enabled GPU you will get the message:
```
<Managed Device 0>
```
If your machine has multiple GPUs, you might want to select which one to use. By default the CUDA driver selects the fastest GPU as the device 0, which is the default device used by Numba.
```python
numba.cuda.select_device( device_id )
```

## Using the CUDA simulator
If you don’t have a CUDA-enabled GPU then you will need to use the CUDA simulator. The simulator is enabled by setting the environment variable NUMBA_ENABLE_CUDASIM to 1 in your system (shell command line).
```shell
export NUMBA_ENABLE_CUDASIM=1
```

In [ ]:
!export NUMBA_ENABLE_CUDASIM=1

In [ ]:
# Listing the GPUs in the system
from numba import cuda
# print(cuda.gpus)

### Writing CUDA kernels
- CUDA has an execution model **unlike the traditional sequential model** used for programming CPUs.  


- In CUDA, the code you write will be **executed by multiple threads at once** (often hundreds or thousands).     

- Your solution will be modeled by defining a **thread hierarchy of grid, blocks, and threads**.      


See [https://nyu-cds.github.io/python-gpu/02-cuda/](https://nyu-cds.github.io/python-gpu/02-cuda/)

### Kernel declaration
A kernel function is a GPU function that is meant to be called from CPU code. It has two fundamental characteristics:

- kernels cannot explicitly return a value; **all result data must be written to an array passed to the function** (if computing a scalar, you have to pass a one-element array);
- kernels explicitly declare their **thread hierarchy** when called: i.e. **the number of blocks per grid** and **the number of threads per block** (note that while a kernel is compiled once, it can be called multiple times with different block sizes or grid sizes).



Numba also exposes three kinds of GPU memory:  
- global device memory  
- shared memory  
- local memory   

<img src='http://docs.nvidia.com/cuda/parallel-thread-execution/graphics/memory-hierarchy.png'>

It is important that you carefully consider how to use and access memory in order to minimize bandwidth requirements and contention.

NVIDIA suggests the following recommendations to achieve the best performance:
- Find ways to parallelize sequential code
- Minimize data transfers between the host and the device
- Adjust kernel launch configuration to maximize device utilization
- Ensure global memory accesses are coalesced (neighboring threads access contiguous memory locations)
- Minimize redundant accesses to global memory whenever possible
- Avoid different execution paths within the same warp (warp is a number of threads that can be executed concurrently)

### Kernel

In [ ]:
from numba import cuda

@cuda.jit
def my_kernel(io_array):
    # code here
    io_array

### Kernel invocation
A kernel is typically launched in the following way:

In [ ]:
import numpy as np

# Create the data array here is a simple example and 
# the data is usually initialized in some other way
data = np.ones(256)

# Set the number of threads in a block
threadsperblock = 32 

# Calculate the number of thread blocks in the grid
blockspergrid = (data.size + (threadsperblock - 1)) // threadsperblock

# or calculate this way:
blockspergrid = int(np.ceil(data.size / threadsperblock))
# here it is 256 / 32 = 8

# Now start the kernel
my_kernel[blockspergrid, threadsperblock](data)

# Print the result
print(data)

### Choosing the block size
The two-level thread hierarchy is important for the following reasons:

- On the software side, the block size determines how many threads share a given area of shared memory.
- On the hardware side, the block size must be large enough for full occupation of execution units; recommendations can be found in the CUDA C Programming Guide. https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation

The block size you choose depends on a range of factors, including:

- The size of the data array
- The size of the shared memory per block (e.g. 64KB)
- The maximum number of threads per block supported by the hardware (e.g. 512 or 1024)
- The maximum number of threads per multiprocessor (MP) (e.g. 2048)
- The maximum number of blocks per MP (e.g. 32)
- The number of threads that can be executed concurrently (a “warp” i.e. 32)

The execution of threads in a warp has a big effect on the computational throughput. 

- If all threads in a warp are **executing the same instruction** then they **can all be executed in parallel**. 

- But if one or more threads is **executing a different instruction**, the warp has to be **split into groups** of threads, and these groups execute serially.

**Rules of thumb for threads per block:**

- Should be a round multiple of the warp size (32)
- A good place to start is 128-512 but benchmarking is required to determine the optimal value.


### Thread positioning

- When running a kernel, the kernel function’s code is executed by every thread once. Therefore **the kernel in each thread needs know which array element(s) it is responsible for**; see output[threadID] below.  


- More complex algorithms may define more complex responsibilities, but the underlying principle is the same.  


- To help deal with multi-dimensional arrays, CUDA allows you to specify **multi-dimensional blocks and grids**.
Blockspergrid and threadsperblock can be 1-dim, 2-dim, or 3-dim (tuples of one, two or three integers). 


---

<img src="02-threadmapping.png" alt="Drawing" style="width: 400px;"/>

In [ ]:
@cuda.jit
def my_kernel(io_array):
    
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    
    # Block id in a 1D grid
    bx = cuda.blockIdx.x
    
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    
    # Compute flattened index inside the array
    pos = tx + bx * bw
    
    if pos < io_array.size: # Check array boundaries
        io_array[pos] = 2 * pos # do some computation;

---


<img src="concepts.png" alt="Drawing" style="width: 800px;"/>


In [ ]:
import numpy as np

data = np.zeros(256)

threadsperblock = 32 
blockspergrid = int(np.ceil(data.size / threadsperblock)) # here it is 8

my_kernel[blockspergrid, threadsperblock](data)

print(data)

### Absolute positions
Simple algorithms will tend to always use thread indices in the same way as shown in the example above. Numba provides additional facilities to automate such calculations:

**numba.cuda.grid(ndim)**:  Return the absolute position of the current thread in the entire grid of blocks. _ndim_ should correspond to the number of dimensions declared when instantiating the kernel. If ndim is 1, a single integer is returned. If ndim is 2 or 3, a tuple of the given number of integers is returned.

**numba.cuda.gridsize(ndim)**: Return the absolute size (or shape) in threads of the entire grid of blocks. _ndim_ has the same meaning as in grid() above.

In [ ]:
@cuda.jit
def my_kernel2(io_array):
    
    pos = cuda.grid(1)
    
    if pos < io_array.size:
        io_array[pos] = 2 * pos

In [ ]:
import numpy as np

data = np.zeros(256)

threadsperblock = 32
blockspergrid = int(np.ceil(data.size / threadsperblock))

my_kernel2[blockspergrid, threadsperblock](data)

print(data)

---

### Real Example: Matrix Multiplication
The following code sample is a straightforward implementation of matrix multiplication for matrices where each thread reads one row of A and one column of B and computes the corresponding element of C. For input arrays where A.shape == (m, n) and B.shape == (n, p) then the result shape will be C.shape = (m, p).
<img src="05-matmul.png" alt="Drawing" style="width: 400px;"/>

In [ ]:
# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    each thread computes one element of C
    """
    row, col = cuda.grid(2)
    
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.0
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

### CUDA Programming Model Basics

[https://devblogs.nvidia.com/easy-introduction-cuda-c-and-c/](https://devblogs.nvidia.com/easy-introduction-cuda-c-and-c/)


Given the heterogeneous nature of the CUDA programming model, a typical sequence of operations for a CUDA C program is:

- Declare and allocate host and device memory.  
- Initialize host data.  
- Transfer data from the host to the device.  
- Execute one or more kernels.  
- Transfer results from the device to the host.  


In [ ]:
# Host code
from numba import cuda
import numpy 

# Initialize the data arrays
A = numpy.full((24, 16), 3, numpy.float) # matrix containing all 3's
B = numpy.full((16, 24), 4, numpy.float) # matrix containing all 4's

# Copy the arrays to the device (GPU)
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)

# Allocate memory on the device (GPU) for the result
# C_global_mem = cuda.device_array((24, 24), dtype=numpy.float)
C_global_mem = cuda.device_array((24, 24))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(numpy.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(numpy.ceil(B.shape[1] / threadsperblock[1]))
# two dimenesional block-per-grid
blockspergrid = (blockspergrid_x, blockspergrid_y)
print("blockspergrid: ", blockspergrid)

# Start the kernel 
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

# Copy the result back to the host
C = C_global_mem.copy_to_host()

print("C=\n", C)

A problem with this code is that each thread is reading from the global memory containing the copies of A and B. In fact, the A global memory is read B.shape[1] times and the B global memory is read A.shape[0] times. Since global memory is fairly slow, this results in an inefficient use of the GPU.

Solution: use **shared memory**! (more in your lab)
* copy small blocks of the matrix into shared memory
* synchronize threads to wait until all threads have access to this new shared memory
* compute partial sums only using the small block

<img src='https://nyu-cds.github.io/python-numba/fig/05-matmulshared.png' style="width: 400px;">

In [ ]:
from numba import cuda, float32
import numpy
import math

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

# The data array
A = numpy.full((TPB*2, TPB*3), 3, numpy.float) # [32 x 48] matrix containing all 3's
B = numpy.full((TPB*3, TPB*1), 4, numpy.float) # [48 x 16] matrix containing all 4's

A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.device_array((TPB*2, TPB*1)) # [32 x 16] matrix result

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
res = C_global_mem.copy_to_host()

print(res)